In [4]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline

# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from bs4 import BeautifulSoup
from collections import OrderedDict # provides the ordered dictionary
import re # for regular expressions used below
import urllib # to read from URLs
import json
import networkx as nx # network analysis
from networkx.readwrite import json_graph
import itertools
import os.path
from datetime import datetime # for time measurement
import sys
import os
import pickle
import subprocess as subp
import gzip

from jellyfish import jaro_distance, jaro_winkler, hamming_distance, levenshtein_distance
import scipy.cluster.hierarchy as scipycluster

from skimage import io, exposure
from scipy.spatial import distance
# import the k-means algorithm
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import pairwise_distances_argmin,pairwise_distances_argmin_min, pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer

# OAI
from sickle import Sickle

import googlemaps

def printLog(text):
    now=str(datetime.now())
    print "["+now+"]\t"+text
    # forces to output the result of the print command immediately, see: http://stackoverflow.com/questions/230751/how-to-flush-output-of-python-print
    sys.stdout.flush()
    
def pickleCompress(fileName,pickledObject):
    printLog("Pickling to '%s'" %fileName)
    f = gzip.open(fileName,'wb')
    pickle.dump(pickledObject,f)
    f.close()
    printLog("Pickling done.")
    
def pickleDecompress(fileName):
    #restore the object
    printLog("Depickling from '%s'" %fileName)
    f = gzip.open(fileName,'rb')
    pickledObject = pickle.load(f)
    f.close()
    printLog("Depickling done.")
    return pickledObject

In [2]:
df=pickleDecompress('clean_dataframe_with_century.picklez')

[2016-03-03 21:29:07.970434]	Depickling from 'clean_dataframe_with_century.picklez'
[2016-03-03 21:29:43.428282]	Depickling done.


In [14]:
tfidfvectorizer = TfidfVectorizer(min_df=1)
df2=df[df.title.notnull()]
corpus=df2.title.astype(str)
Xtfidf=tfidfvectorizer.fit_transform(corpus)
wordstfidf=tfidfvectorizer.get_feature_names()

In [24]:
true_k=100
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
# fit the k-means algorithm on the data created above
km.fit(Xtfidf)
# add the detected clusters as a new column to the original data frame
df2['cluster']=km.labels_
# group the data by the cluster and describe it
df2.groupby('cluster').describe()

century     latitude    longitude
cluster                                             
0       count   179.000000   106.000000   106.000000
        mean     16.810056    48.239131    13.924442
        std       0.568568    11.457401    20.169983
        min      15.000000   -30.766253   -74.072092
        25%      17.000000    48.370545    10.649284
        50%      17.000000    51.107885    13.137871
        75%      17.000000    52.489032    14.550567
        max      19.000000    59.934280   106.629664
1       count   482.000000   374.000000   374.000000
        mean     16.946058    50.686737    11.225477
        std       0.790533     5.188191    12.237453
        min      14.000000   -16.748080  -101.633797
        25%      17.000000    50.080869    10.566448
        50%      17.000000    51.370736    12.373075
        75%      17.000000    52.520007    13.404954
        max      19.000000    56.949649    77.669182
2       count  1654.000000  1149.000000  1149.000000
        mean     16.645707    49.881537    13.337174
        std       2.916086     5.880584    24.902171
        min      -1.000000    -7.813808  -149.268221
        25%      17.000000    48.401082     9.203804
        50%      17.000000    51.339695    12.373075
        75%      17.000000    52.520007    14.550567
        max      19.000000    62.961944   136.906398
3       count  1318.000000   965.000000   965.000000
        mean     16.650228    50.345611    11.970058
        std       1.669266     4.579022    14.959200
        min      -1.000000    -1.266667  -121.457523
        25%      17.000000    48.856614     9.993682
        50%      17.000000    51.339695    12.373075
...                    ...          ...          ...
96      std       2.909377     5.763960    25.563999
        min      -1.000000     2.189594  -101.633797
        25%      17.000000    48.741355     9.203804
        50%      17.000000    51.339695    12.373075
        75%      17.000000    52.520007    13.737262
        max      19.000000    59.995930   139.888320
97      count   551.000000   424.000000   424.000000
        mean     16.145191    50.267907    12.508652
        std       1.429803     3.478974    10.229474
        min      -1.000000     8.351840    -7.872160
        25%      15.000000    48.573405    10.504825
        50%      16.000000    50.984768    11.627624
        75%      17.000000    52.184926    13.404954
        max      19.000000    59.860686   136.906398
98      count   244.000000   206.000000   206.000000
        mean     17.479508    51.438999    10.642765
        std       0.978992     1.789184    13.622517
        min      15.000000    43.060210  -149.268221
        25%      17.000000    50.955062    10.530289
        50%      18.000000    52.120533    12.373075
        75%      18.000000    52.520007    13.404954
        max      19.000000    61.557019    18.646638
99      count   582.000000   422.000000   422.000000
        mean     16.773196    50.887826    10.498469
        std       1.519841     2.670617    10.083789
        min      -1.000000    18.220833   -87.629798
        25%      17.000000    49.452030     9.993682
        50%      17.000000    51.339695    11.896572
        75%      17.000000    52.520007    13.404954
        max      19.000000    56.949649    24.105186

[800 rows x 3 columns]

In [27]:
df2=df2.sort("cluster")
df2[['title','cluster']].tail(100)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,title,cluster
7226,Umschlag mit der Aufschrift von Humboldts Hand...,99
91259,Glückwünschungs-Ode an den Hochwohlgebohrnen H...,99
16264,Bums - schon wieder ein Brummer!,99
115923,Theses oder Grundsätze von besondern Kauffmann...,99
72408,Brief von Adelbert von Chamisso an Antonie von...,99
98357,Die billige Trauer Bey dem schmertzlichen Hint...,99
51322,"Wo soll ich mich hinwenden, Bey der betrübten ...",99
38548,Copey und Abschrifft Des Schreibens/ Der Drey ...,99
84340,Iudicium Mundi Et Spiritus Sancti De Obitu Ius...,99
110257,Kriegs-Rath Vom Belial Und Seinen lieben getre...,99
